<a href="https://colab.research.google.com/github/JesusATL/pc5-Atencion-sobre-lenguaje-matematico-usando-transformers/blob/main/Algebra-B%C3%A1sica-Mar2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## En el presente cuaderno se prueba algunas predicciones a partir del repositorio https://github.com/andrewschreiber/hs-math-nlp/, en su rama master en la fecha marzo del 2020, para asegurar el buen funcionamiento de algunos cuadernos de referencia en el repositorio.

### Instalamos el requirements.txt

In [ ]:
!pip install absl-py==0.9.0
!pip install astroid==2.4.1
!pip install cachetools==4.1.0
!pip install certifi==2020.4.5.1
!pip install chardet==3.0.4
!pip install future==0.18.2
!pip install google-api-core==1.17.0
!pip install google-auth==1.16.0
!pip install google-auth-oauthlib==0.4.1
!pip install google-cloud-core==1.3.0
!pip install google-cloud-storage==1.28.1
!pip install google-resumable-media==0.5.0
!pip install googleapis-common-protos==1.51.0
!pip install grpcio==1.29.0
!pip install idna==2.9
!pip install importlib-metadata==1.6.0
!pip install isort==4.3.21
!pip install lazy-object-proxy==1.4.3
!pip install Markdown==3.2.2
!pip install mccabe==0.6.1
!pip install numpy==1.18.4
!pip install oauthlib==3.1.0
!pip install pandas==1.0.3
!pip install protobuf==3.12.2
!pip install pyasn1==0.4.8
!pip install pyasn1-modules==0.2.8
!pip install pylint==2.5.2
!pip install python-dateutil==2.8.1
!pip install pytz==2020.1
!pip install requests==2.23.0
!pip install requests-oauthlib==1.3.0
!pip install rsa==4.0
!pip install six==1.15.0
!pip install tensorboard==2.2.2
!pip install tensorboard-plugin-wit==1.6.0.post3
!pip install tensorboardX==2.0
!pip install toml==0.10.0
!pip install torch==1.4.0
!pip install tqdm==4.46.0
!pip install typed-ast==1.4.1
!pip install urllib3==1.25.9
!pip install Werkzeug==1.0.1
!pip install wrapt==1.11.2
!pip install zipp==3.1.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Restauramos el repositorio en Marzo del 2020

In [ ]:
!git checkout -b mar2020 f8170b36fbd2392baa1e8f41d0a0d06efdd6e0e2

fatal: A branch named 'mar2020' already exists.


### Nos movemos a la carpeta del repositorio

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/transformer/hs-math-nlp')

### Importamos las librerías necesarias

In [ ]:
import math
import numpy as np
import torch
from torch.utils import data
import torch.optim as optim
import torch.nn.functional as F
from tensorboard_utils import Tensorboard

from math_dataset import (
    LazyFileMathDataset,
    MathDatasetManager,
    question_answer_to_position_batch_collate_fn
)

import model_process
import utils
import checkpoints

from tensorboard_utils import Tensorboard
from tensorboard_utils import tensorboard_event_accumulator

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook  

print("Torch Version", torch.__version__)

%reload_ext autoreload
%autoreload 2

Torch Version 1.4.0


In [ ]:
device = torch.device("cuda")
print("device", device)

device cuda


### Observamos los modulos con los que se puede trabajar 

In [ ]:
mdsmgr = MathDatasetManager("/content/drive/MyDrive/Colab Notebooks/transformer/mathematics_dataset-v1.0")

initialized MultiFilesMathDataset with categories ['numbers', 'polynomials', 'comparison', 'probability', 'algebra', 'arithmetic', 'measurement', 'calculus'] and types ['train-easy', 'train-medium', 'train-hard', 'interpolate', 'extrapolate']


In [ ]:
model_filepath = "/content/drive/MyDrive/Colab Notebooks/transformer/hs-math-nlp/checkpoints/checkpoint_felipe.pth"

In [ ]:
print("tipos", list(mdsmgr.get_types()))

types ['train-easy', 'train-medium', 'train-hard', 'interpolate', 'extrapolate']


In [ ]:
print("categorias", list(mdsmgr.get_categories()))

categories ['numbers', 'polynomials', 'comparison', 'probability', 'algebra', 'arithmetic', 'measurement', 'calculus']


In [ ]:
print("modulos de aritmética", mdsmgr.get_modules_for_category('arithmetic'))

modules of arithmetic dict_keys(['mul', 'add_or_sub_in_base', 'mixed', 'mul_div_multiple', 'add_sub_multiple', 'add_or_sub', 'simplify_surd', 'nearest_integer_root', 'div', 'add_or_sub(1)', 'mul_div_multiple_longer', 'add_or_sub_big', 'add_sub_multiple_longer', 'div_big', 'mixed_longer', 'mul_big'])


In [ ]:
ds = mdsmgr.build_dataset_from_module('arithmetic', 'add_or_sub', 'train-easy')
print("tamaño", len(ds))

size 666666


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
ds = mdsmgr.build_dataset_from_module('arithmetic', 'add_or_sub', 'train-easy', max_elements=1000)
print("tamaño", len(ds))

size 1000


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
ds = mdsmgr.build_dataset_from_modules('arithmetic', ['add_or_sub', 'add_sub_multiple'], 'train-easy')
print("tamaño", len(ds))

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


size 1333332


In [ ]:
ds = mdsmgr.build_dataset_from_category('arithmetic', 'train-easy')
print("tamaño", len(ds))

adding category arithmetic/../train-easy
attempting to add module arithmetic/mul/train-easy


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


added module arithmetic/mul/train-easy
attempting to add module arithmetic/add_or_sub_in_base/train-easy
added module arithmetic/add_or_sub_in_base/train-easy
attempting to add module arithmetic/mixed/train-easy
added module arithmetic/mixed/train-easy
attempting to add module arithmetic/mul_div_multiple/train-easy
added module arithmetic/mul_div_multiple/train-easy
attempting to add module arithmetic/add_sub_multiple/train-easy
added module arithmetic/add_sub_multiple/train-easy
attempting to add module arithmetic/add_or_sub/train-easy
added module arithmetic/add_or_sub/train-easy
attempting to add module arithmetic/simplify_surd/train-easy
added module arithmetic/simplify_surd/train-easy
attempting to add module arithmetic/nearest_integer_root/train-easy
added module arithmetic/nearest_integer_root/train-easy
attempting to add module arithmetic/div/train-easy
added module arithmetic/div/train-easy
size 5999994


In [ ]:
ds = mdsmgr.build_dataset_from_categories(['arithmetic', 'polynomials'], 'train-easy')
print("tamaño", len(ds))

adding category.. arithmetic/../train-easy
attempting to add module arithmetic/mul/train-easy
added module arithmetic/mul/train-easy
attempting to add module arithmetic/add_or_sub_in_base/train-easy
added module arithmetic/add_or_sub_in_base/train-easy
attempting to add module arithmetic/mixed/train-easy
added module arithmetic/mixed/train-easy
attempting to add module arithmetic/mul_div_multiple/train-easy
added module arithmetic/mul_div_multiple/train-easy
attempting to add module arithmetic/add_sub_multiple/train-easy
added module arithmetic/add_sub_multiple/train-easy
attempting to add module arithmetic/add_or_sub/train-easy


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


added module arithmetic/add_or_sub/train-easy
attempting to add module arithmetic/simplify_surd/train-easy
added module arithmetic/simplify_surd/train-easy
attempting to add module arithmetic/nearest_integer_root/train-easy
added module arithmetic/nearest_integer_root/train-easy
attempting to add module arithmetic/div/train-easy
added module arithmetic/div/train-easy
adding category.. polynomials/../train-easy
attempting to add module polynomials/collect/train-easy
added module polynomials/collect/train-easy
attempting to add module polynomials/evaluate/train-easy
added module polynomials/evaluate/train-easy
attempting to add module polynomials/add/train-easy
added module polynomials/add/train-easy
attempting to add module polynomials/evaluate_composed/train-easy
added module polynomials/evaluate_composed/train-easy
attempting to add module polynomials/expand/train-easy
added module polynomials/expand/train-easy
attempting to add module polynomials/simplify_power/train-easy
added modul

In [ ]:
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda")
print("device", device)

device cuda


In [ ]:
exp_name = "algebra"
unique_id = "2020-07-25_0900"

In [ ]:
ds = mdsmgr.build_dataset_from_module(
    'algebra', 'linear_1d', 'train-easy'
)
print("train-easy dataset size", len(ds))

train-easy dataset size 666666


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
ds_interpolate = mdsmgr.build_dataset_from_module(
    'algebra', 'linear_1d', 'interpolate'
)
print("interpolate dataset size", len(ds_interpolate))

interpolate dataset size 10000


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
model = utils.build_transformer()


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=6e-6, betas=(0.9, 0.995), eps=1e-9)

In [ ]:
train_ds, val_ds = math_dataset.random_split_dataset(ds, split_rate=0.9)

#Cargamos el data loader
train_loader = data.DataLoader(
    train_ds, batch_size=128, shuffle=True, num_workers=4,
    collate_fn=question_answer_to_position_batch_collate_fn)

val_loader = data.DataLoader(
    val_ds, batch_size=128, shuffle=False, num_workers=4,
    collate_fn=question_answer_to_position_batch_collate_fn)

interpolate_loader = data.DataLoader(
    ds_interpolate, batch_size=128, shuffle=False, num_workers=4,
    collate_fn=question_answer_to_position_batch_collate_fn)

In [ ]:
tb = Tensorboard(exp_name, unique_name=unique_id)

Writing TensorBoard events locally to runs/algebra_2020-07-25_0900


In [ ]:
model = model.to(device)

In [ ]:
model_process.train(
    exp_name = exp_name, 
    unique_id = unique_id,
    model = model, 
    training_data = train_loader, 
    validation_data = val_loader, 
    interpolate_data = interpolate_loader,
    optimizer = optimizer, 
    device = device,
    epochs=1, 
    tb=tb, 
    log_interval=100,
)

  0%|          | 0/4688 [00:00<?, ?it/s]

[ Epoch 0 ]


[Training]  loss: 1.7305487177632488, ppl:  5.64375, accuracy: 64.464 %, elapse: 1380060.880ms
Save checkpoint time: 4507.014274597168


AttributeError: ignored

In [ ]:
exp_name = "math_ds_algebra_linear_1d_easy"
unique_id = "2019-05-25_0900"
model_filepath = "./checkpoints/math_ds_algebra_linear_1d_easy_2019-05-25_0900_latest_checkpoint.pth"

In [ ]:
# build default transformer model
model = utiMls.build_transformer()
# restore model from checkpoint
_ = checkpoints.restore_checkpoint(model_filepath, model)

Extracting state from ./checkpoints/math_ds_algebra_linear_1d_easy_2019-05-25_0900_latest_checkpoint.pth
Loading model state_dict from state found in ./checkpoints/math_ds_algebra_linear_1d_easy_2019-05-25_0900_latest_checkpoint.pth


In [ ]:
%ls

checkpoints/           math_ds_sandbox.ipynb    spot_startup_script.sh
checkpoints.py         math_ds_train.ipynb      tensorboard_utils.py
gcloud_instance.sh     model_process.py         training.py
loss.py                __pycache__/             transformer/
LSTM/                  README.md                utils.py
math_dataset.py        runs/
math_ds_predict.ipynb  spot_shutdown_script.sh


In [ ]:
valid_ea = tensorboard_event_accumulator(
    "./runs/math_ds_algebra_linear_1d_easy_2019-05-25_0900_train/events.out.tfevents.1626765620.213c2d3b66d6"
)

interpolate_ea = tensorboard_event_accumulator(
    "./runs/math_ds_algebra_linear_1d_easy_2019-05-25_0900_interpolate/events.out.tfevents.1626761358.22b9ae83bd65"
)

valid_accuracy = valid_ea.Scalars("epoch/accuracy")
valid_loss_per_char = valid_ea.Scalars("epoch/loss_per_char")

interpolate_accuracy = interpolate_ea.Scalars("epoch/accuracy")
interpolate_loss_per_char = interpolate_ea.Scalars("epoch/loss_per_char")

In [141]:
questions = [
    "Solve 10 * d - 20 = -10 for d."
]

print(model_process.predict_multiple(questions, model, device))

[{'resp': '1', 'score': -0.0258636474609375}]
